In [33]:
import pandas as pd
import numpy as np
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')
filename = r'table.xls'

In [34]:
# Чтение файла формата xls
sheet_number = 0
xls_frame = pd.ExcelFile(filename).parse(sheet_number)
# Преобразование столбца date из строки в datetime
xls_frame['date'] = pd.to_datetime(xls_frame['date'], format='%Y-%m-%d')
xls_frame

,date,installs,costs,revenue,payers share
0,2021-05-01,5,150,314,5
1,2021-04-24,15,33,159,10
2,2020-12-24,7,421,512,12
3,2020-12-21,3,127,13,7


In [35]:
# Для каждой колонки применяем необходимые функции
funcs = [
         pd.NamedAgg(column='mean', aggfunc=np.mean),
         pd.NamedAgg(column='std', aggfunc=np.std),
         pd.NamedAgg(column='var', aggfunc=st.variation),
         pd.NamedAgg(column='med', aggfunc=np.median),
         pd.NamedAgg(column='q25', aggfunc=lambda x: np.percentile(x, 25)),
         pd.NamedAgg(column='q75', aggfunc=lambda x: np.percentile(x, 75)),
         ]
grouper = [pd.Grouper(key='date', freq='w')]
grouped_df = xls_frame.groupby(grouper).agg(funcs)
grouped_df = grouped_df.loc[(~np.isnan(grouped_df.loc[:, pd.IndexSlice[:, 'mean']])).any(axis=1)].reset_index()
grouped_df

date installs                                   costs              \
                 mean       std  var   med   q25   q75   mean         std   
0 2020-12-27      5.0  2.828427  0.4   5.0   4.0   6.0  274.0  207.889394   
1 2021-04-25     15.0       NaN  0.0  15.0  15.0  15.0   33.0         NaN   
2 2021-05-02      5.0       NaN  0.0   5.0   5.0   5.0  150.0         NaN   

             ...   revenue                        payers share            \
        var  ...       var    med     q25     q75         mean       std   
0  0.536496  ...  0.950476  262.5  137.75  387.25          9.5  3.535534   
1  0.000000  ...  0.000000  159.0  159.00  159.00         10.0       NaN   
2  0.000000  ...  0.000000  314.0  314.00  314.00          5.0       NaN   

                                 
        var   med    q25    q75  
0  0.263158   9.5   8.25  10.75  
1  0.000000  10.0  10.00  10.00  
2  0.000000   5.0   5.00   5.00  

[3 rows x 25 columns]

In [36]:
# нахождение корреляции по группам, сформированным по недельным промежуткам
corr_func = lambda a: np.corrcoef(a['installs'], a['revenue']-a['costs'])[0, 1]
corr_df = pd.DataFrame(xls_frame)
# сперва находим корреляцию к группах и записываем в новую колонку r, затем группируем по дате, оставляя последнее значение группы 
corr_df = corr_df.groupby(grouper).apply(lambda b: b.assign(r=corr_func(b))).groupby(grouper).last()
corr_df = corr_df.loc[(~np.isnan(corr_df)).any(axis=1), :].reset_index()
grouped_df['r'] = corr_df['r']
grouped_df

date installs                                   costs              \
                 mean       std  var   med   q25   q75   mean         std   
0 2020-12-27      5.0  2.828427  0.4   5.0   4.0   6.0  274.0  207.889394   
1 2021-04-25     15.0       NaN  0.0  15.0  15.0  15.0   33.0         NaN   
2 2021-05-02      5.0       NaN  0.0   5.0   5.0   5.0  150.0         NaN   

             ... revenue                 payers share                      \
        var  ...     med     q25     q75         mean       std       var   
0  0.536496  ...   262.5  137.75  387.25          9.5  3.535534  0.263158   
1  0.000000  ...   159.0  159.00  159.00         10.0       NaN  0.000000   
2  0.000000  ...   314.0  314.00  314.00          5.0       NaN  0.000000   

                         r  
    med    q25    q75       
0   9.5   8.25  10.75  1.0  
1  10.0  10.00  10.00  NaN  
2   5.0   5.00   5.00  NaN  

[3 rows x 26 columns]

In [37]:
with pd.ExcelWriter('out_table.xls', mode='w') as writer:
    grouped_df.to_excel(writer, sheet_name='Sheet 1')